### Este programa analiza las subredes corruptas guardadas como ".graphml"  y calcula sus métricas

(Este código guarda y analiza las redes corruptas en formato ".graphml" y calcula sus métricas, pero grafica mal los titulos y no grafica las redes corruptas)

In [ ]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scienceplots
from networkx.algorithms import community
import os
import time
import sys
import re # Para sanitizar nombres de archivo
from collections import Counter

try:
    from IPython.display import SVG, display, Image
except ImportError:
    SVG, display, Image = None, None, None

# --- Parámetros Juego ---
PARAMS_DINAMARCA = {'r': 2.5, 's': 1, 't': 3, 'p': 8, 'c': 0.2, 'q': 11, 'd': 0.5}
PARAMS_SINGAPUR =  {'r': 2, 's': 1, 't': 4, 'p': 10, 'c': 0.2, 'q': 15, 'd': 0.2}
PARAMS_MEXICO =    {'r': 2, 's': 1, 't': 4, 'p': 10, 'c': 2, 'q': 5, 'd': 4}

# ---------------------------------------------------------------------------
# --- INICIO DE FUNCIONES DE SIMULACIÓN (SIN CAMBIOS SIGNIFICATIVOS) ---
# ---------------------------------------------------------------------------
def calcular_pago_individual_corrupcion(strat_i, strat_j, params):
    r, s, t, p, c, q, d = params['r'], params['s'], params['t'], params['p'], params['c'], params['q'], params['d']
    payoffs = { ('C', 'C'): r, ('C', 'D'): -s, ('C', 'H'): r, ('C', 'K'): -s, ('D', 'C'): t, ('D', 'D'): 0, ('D', 'H'): t-p, ('D', 'K'): -p, ('H', 'C'): r, ('H', 'D'): -s-c, ('H', 'H'): r, ('H', 'K'): -s-d, ('K', 'C'): t, ('K', 'D'): -c, ('K', 'H'): t-q, ('K', 'K'): -q-d }
    return payoffs.get((strat_i, strat_j), 0)

def actualizar_estrategia_corrupcion_red_homofilia(red, nodo, estrategias, recompensas_totales, K, homophily_strength):
    estrategia_actual = estrategias[nodo]; recompensa_total_actual = recompensas_totales[nodo]
    vecinos = list(red.neighbors(nodo))
    if not vecinos: return estrategia_actual
    vecino_elegido = random.choice(vecinos); estrategia_vecino = estrategias[vecino_elegido]; recompensa_total_vecino = recompensas_totales[vecino_elegido]
    probabilidad_adoptar_base = 0.0
    try:
        if K <= 1e-9: probabilidad_adoptar_base = 1.0 if recompensa_total_vecino > recompensa_total_actual else 0.0
        else: delta_recompensa_scaled = np.clip((recompensa_total_actual - recompensa_total_vecino) / K, -700, 700); probabilidad_adoptar_base = 1 / (1 + np.exp(delta_recompensa_scaled))
    except (OverflowError, FloatingPointError): probabilidad_adoptar_base = 1.0 if recompensa_total_vecino > recompensa_total_actual else 0.0
    if estrategia_actual == estrategia_vecino: probabilidad_adoptar_final = probabilidad_adoptar_base * (1 + homophily_strength)
    else: probabilidad_adoptar_final = probabilidad_adoptar_base * (1 - homophily_strength)
    probabilidad_adoptar_final = np.clip(probabilidad_adoptar_final, 0.0, 1.0)
    return estrategia_vecino if random.random() < probabilidad_adoptar_final else estrategia_actual

def calcular_centralidad(red):
    try: centrality = nx.eigenvector_centrality_numpy(red); print("  Usando centralidad de Eigenvector.")
    except Exception as e: print(f"  Advertencia: Eigenvector falló ({e}). Usando Grado."); centrality = nx.degree_centrality(red)
    return centrality

def seleccionar_nodos_cm(centrality_dict, cm_size, placement_type):
    cm_size = int(round(cm_size))
    if not 0 <= cm_size <= len(centrality_dict): raise ValueError(f"CM size ({cm_size}) inválido.")
    if cm_size == 0: return []
    if cm_size == len(centrality_dict): return list(centrality_dict.keys())
    reverse_sort = (placement_type == 'Nodos centrales')
    nodos_ordenados = sorted(centrality_dict, key=centrality_dict.get, reverse=reverse_sort)
    if placement_type in ['Nodos centrales', 'Nodos perifericos']: cm_nodes = nodos_ordenados[:cm_size]
    elif placement_type == 'Nodos aleatorios': cm_nodes = random.sample(list(centrality_dict.keys()), cm_size)
    else: raise ValueError(f"Colocación CM desconocida: {placement_type}")
    return cm_nodes

def inicializar_estrategias_cm_flexible(n, cm_nodes, cm_strategies, non_cm_strategies, seed=None):
    if seed is not None: random.seed(seed)
    estrategias = {}; non_cm_nodes = list(set(range(n)) - set(cm_nodes))
    if not cm_strategies and cm_nodes : raise ValueError("cm_strategies vacía pero cm_nodes no lo está.")
    if not non_cm_strategies and non_cm_nodes: raise ValueError("non_cm_strategies vacía pero non_cm_nodes no lo está.")
    random.shuffle(cm_nodes); random.shuffle(non_cm_nodes)
    if cm_strategies:
        for i, nodo in enumerate(cm_nodes): estrategias[nodo] = cm_strategies[i % len(cm_strategies)]
    if non_cm_strategies:
        for i, nodo in enumerate(non_cm_nodes): estrategias[nodo] = non_cm_strategies[i % len(non_cm_strategies)]
    return dict(sorted(estrategias.items()))

def ejecutar_simulacion_corrupcion(num_nodos=100, network_type='Completa', network_params=None, cm_size_actual=50, cm_placement_type='Nodos aleatorios', cm_strategies = ['C', 'H'], non_cm_strategies = ['D', 'K'], params_juego=PARAMS_MEXICO, K=0.1, homophily_strength=0.0, rondas=15, seed=None):
    if seed is not None: random.seed(seed); np.random.seed(seed)
    print(f"Creando red tipo '{network_type}'...")
    network_params_str_out = ""
    if network_type == 'Completa': red = nx.complete_graph(num_nodos)
    elif network_type == 'Barabási-Albert':
        if network_params is None: network_params = {}
        m = network_params.get('m', 2)
        if m >= num_nodos: m = max(1, num_nodos // 2)
        red = nx.barabasi_albert_graph(num_nodos, m, seed=seed); network_params_str_out = f" m={m}"
    else: raise ValueError(f"Tipo de red desconocido: {network_type}")
    print(f"Red creada: {red.number_of_nodes()} nodos, {red.number_of_edges()} enlaces.")
    print(f"Calculando centralidad..."); centrality = calcular_centralidad(red)
    print(f"Seleccionando {cm_size_actual} nodos CM ({cm_placement_type})..."); cm_nodes = seleccionar_nodos_cm(centrality, cm_size_actual, cm_placement_type)
    print(f"Inicializando estrategias: CM={cm_strategies}, No-CM={non_cm_strategies} con seed={seed}...")
    estrategias = inicializar_estrategias_cm_flexible(num_nodos, cm_nodes, cm_strategies, non_cm_strategies, seed=seed)
    counts_init = Counter(estrategias.values()); print(f"  Estrategias iniciales: {counts_init}")
    historia_fracciones = {'C': [], 'D': [], 'H': [], 'K': []}; nodos_lista = list(red.nodes())
    print(f"Iniciando {rondas} rondas...");
    for r in range(rondas):
        recompensas_totales = {nodo: 0 for nodo in nodos_lista}
        for nodo_i in nodos_lista:
            strat_i = estrategias[nodo_i]; vecinos_i = list(red.neighbors(nodo_i))
            if not vecinos_i: continue
            for nodo_j in vecinos_i: strat_j = estrategias[nodo_j]; recompensas_totales[nodo_i] += calcular_pago_individual_corrupcion(strat_i, strat_j, params_juego)
        siguientes_estrategias = {}; nodos_actualizar = nodos_lista[:]; random.shuffle(nodos_actualizar)
        for nodo in nodos_actualizar: siguientes_estrategias[nodo] = actualizar_estrategia_corrupcion_red_homofilia(red, nodo, estrategias, recompensas_totales, K, homophily_strength)
        estrategias = siguientes_estrategias
        conteo_actual = Counter(estrategias.values())
        for strat_key in ['C', 'D', 'H', 'K']: fraccion = conteo_actual.get(strat_key, 0) / num_nodos if num_nodos > 0 else 0; historia_fracciones[strat_key].append(fraccion)
    print("Simulación completada.")
    return historia_fracciones, red, estrategias

# --- Bloque de Funciones de Análisis (Integrado y Modificado) ---
class Logger:
    def __init__(self, filename="log.txt"):
        self.terminal = sys.stdout; self.log = open(filename, "w", encoding='utf-8')
    def write(self, message): self.terminal.write(message); self.log.write(message)
    def flush(self): self.terminal.flush(); self.log.flush()
    def close(self): self.log.close()

def analizar_mostrarRed(G, output_path):
    plt.style.use('science'); plt.figure(figsize=(3, 3))
    try: pos = nx.spring_layout(G, seed=42, k=0.6/np.sqrt(G.number_of_nodes()) if G.number_of_nodes()>0 else 0.1)
    except: pos = nx.circular_layout(G)
    nx.draw(G, pos, node_size=30, node_color='#9FA3D5', edge_color='#3B4285', width=0.5, with_labels=False)
    plt.title('Visualización Subred Corrupta (K)'); plt.savefig(f"{output_path}_visualizacion.png", dpi=300, bbox_inches='tight'); plt.close()

def analizar_distribucionDeGrado(G, output_path):
    if not G.nodes() or not G.edges(): print("  - Distribución Grado: Subred sin nodos/aristas."); return
    degrees = [d for n, d in G.degree()]; unique_degrees, counts = np.unique(degrees, return_counts=True); probs = counts / len(G)
    non_zero_probs = probs[probs > 0]; non_zero_degrees = unique_degrees[probs > 0]
    valid_log = non_zero_degrees > 0; gamma = np.nan
    if np.any(valid_log) and len(non_zero_degrees[valid_log]) > 1:
        log_probs = np.log(non_zero_probs[valid_log]); log_degrees = np.log(non_zero_degrees[valid_log])
        slope, intercept, _, _, _ = linregress(log_degrees, log_probs); gamma = round(slope, 2)
        plt.style.use('science'); plt.figure(figsize=(3, 3))
        plt.loglog(non_zero_degrees, non_zero_probs, 'o', color='#9FA3D5', markersize=3)
        plt.plot(non_zero_degrees[valid_log], np.exp(intercept + slope * np.log(non_zero_degrees[valid_log])), '--', label=f'Ajuste: {gamma}', color='#3B4285', linewidth=1)
        plt.xlabel('Grado (k) - Escala Log'); plt.ylabel('P(k) - Escala Log'); plt.legend(fontsize='small')
        plt.savefig(f"{output_path}_dist_grado_log.png", dpi=300, bbox_inches='tight'); plt.close()
    else: print("  - Distribución Grado: No hay suficientes puntos para ajuste.")
    print(f"  - Exponente ley de potencias (gamma): {gamma}")

def analizar_estructuraComunidades(G):
    if not G.nodes() or not G.edges(): print("  - Comunidades: Subred sin nodos/aristas."); return G.copy()
    try: comunidades = list(community.louvain_communities(G, seed=42))
    except: print("  - Comunidades: Error en algoritmo Louvain."); return G.copy()
    num_comunidades = len(comunidades); nodos_comunidad = [len(c) for c in comunidades]; promedio_nodos = round(np.mean(nodos_comunidad), 2) if nodos_comunidad else 0
    print(f"  - Número de comunidades: {num_comunidades}"); print(f"  - Promedio de nodos por comunidad: {promedio_nodos}")
    if num_comunidades > 0:
        max_comunidad_size = max(nodos_comunidad); print(f"  - Comunidad más grande: {max_comunidad_size} nodos")
        return G.subgraph(comunidades[np.argmax(nodos_comunidad)]).copy()
    return G.copy()

def analizar_mundoPequeno(G, G_max_comunidad):
    if not G.nodes(): print("  - Mundo Pequeño: Subred vacía."); return
    clustering_coeff = round(nx.average_clustering(G), 3); avg_path_len = np.nan
    if G_max_comunidad.nodes() and nx.is_connected(G_max_comunidad):
        avg_path_len = round(nx.average_shortest_path_length(G_max_comunidad), 3)
    print(f"  - Coeficiente de agrupamiento global: {clustering_coeff}"); print(f"  - Camino medio más corto (comp. gigante): {avg_path_len}")

def analizar_asortatividad(G, G_max_comunidad):
    if not G.edges(): print("  - Asortatividad: Subred sin aristas."); return
    assort_global = round(nx.degree_assortativity_coefficient(G), 3); assort_local = np.nan
    if G_max_comunidad.edges(): assort_local = round(nx.degree_assortativity_coefficient(G_max_comunidad), 3)
    print(f"  - Asortatividad global: {assort_global}"); print(f"  - Asortatividad local (comp. gigante): {assort_local}")

def analizar_densidad(G, G_max_comunidad):
    if not G.nodes(): print("  - Densidad: Subred vacía."); return
    density_global = round(nx.density(G), 4); density_local = np.nan
    if G_max_comunidad.nodes(): density_local = round(nx.density(G_max_comunidad), 4)
    print(f"  - Densidad global: {density_global}"); print(f"  - Densidad local (comp. gigante): {density_local}")

def analizar_subred_k(subgrafo_k, output_path_base):
    if not isinstance(subgrafo_k, nx.Graph) or not subgrafo_k.nodes(): print("  Análisis de métricas omitido: Subred K vacía."); return
    os.makedirs(os.path.dirname(output_path_base), exist_ok=True)
    log_file = Logger(f"{output_path_base}_metricas.txt"); original_stdout = sys.stdout; sys.stdout = log_file
    print(f"--- ANÁLISIS DE SUBRED K: {os.path.basename(output_path_base)} ---"); print(f"Nodos: {subgrafo_k.number_of_nodes()}, Enlaces: {subgrafo_k.number_of_edges()}"); print("-" * 20)
    G_max_comunidad = analizar_estructuraComunidades(subgrafo_k); print("-" * 20)
    analizar_distribucionDeGrado(subgrafo_k, output_path_base); print("-" * 20)
    analizar_mundoPequeno(subgrafo_k, G_max_comunidad); analizar_asortatividad(subgrafo_k, G_max_comunidad); analizar_densidad(subgrafo_k, G_max_comunidad)
    print("-" * 20); print("--- FIN DE ANÁLISIS ---")
    sys.stdout = original_stdout; log_file.close(); print(f"  Análisis de métricas de subred K guardado en: {log_file.log.name}")
    analizar_mostrarRed(subgrafo_k, output_path_base)

# --- Funciones de Graficado (principales) ---
def graficar_evolucion_corrupcion(historia_fracciones, titulo, network_type_str, homophily_str, cm_title_str, filename):
    plt.style.use(['science', 'ieee']); plt.figure(figsize=(2.5, 2.5))
    colores = {'C': '#DEDEEE', 'D': '#9B9ACA', 'H': '#7976B8', 'K': '#201F3D'}
    estilos = {'C': '-', 'D': '--', 'H': ':', 'K': '-.'}; nombres_legenda = {'C': '(C) C. Coop', 'D': '(D) C. no Coop', 'H': '(H) P. Honrado', 'K': '(K) P. Corrupto'}
    rondas_totales = len(historia_fracciones['C']); eje_x = range(rondas_totales)
    for strat in ['C', 'D', 'H', 'K']: plt.plot(eje_x, historia_fracciones[strat], label=nombres_legenda[strat], color=colores[strat], linestyle=estilos[strat], linewidth=1)
    plt.title(f"Evolución de {titulo}\n{network_type_str} - Homofilia: {int(float(homophily_str) * 100)}\\% \n{cm_title_str}")
    plt.ylim(0, 1.0); plt.xlim(0, rondas_totales -1 if rondas_totales > 1 else 1); plt.legend(fontsize=7, loc='best'); plt.grid(True, linestyle='--', alpha=0.6)
    os.makedirs(os.path.dirname(filename), exist_ok=True); plt.savefig(filename, format="svg", dpi=300, bbox_inches='tight'); print(f"Gráfica de evolución guardada como '{filename}'"); plt.close()

def graficar_red_completa(red, estrategias, pos, titulo, output_path):
    if not isinstance(red, nx.Graph) or not red.nodes(): return
    plt.style.use(['science', 'ieee']); plt.figure(figsize=(2.5, 2.5)); n = red.number_of_nodes()
    color_map = {'C': '#DEDEEE', 'D': '#9B9ACA', 'H': '#7976B8', 'K': '#201F3D', 'Unknown': '#808080'}
    node_colors = [color_map.get(estrategias.get(n, 'Unknown'), '#808080') for n in red.nodes()]
    node_size = 10 if n > 200 else 15; edge_alpha = 0.03 if n > 100 else 0.5; edge_width = 0.05 if n > 100 else 0.3
    nx.draw(red, pos, node_size=node_size, node_color=node_colors, edge_color="#A7A7A7", alpha=0.8, width=edge_width, with_labels=False)
    plt.title(titulo); plt.axis('off')
    plt.savefig(f"{output_path}_red_final.svg", dpi=300, bbox_inches='tight'); plt.close()

def graficar_red_final(red, estrategias_finales, titulo, network_type, network_params_str, homophily_str, cm_title_str, filename):
    if not isinstance(red, nx.Graph) or not red.nodes(): return None
    plt.style.use(['science', 'ieee']); fig_size = (3.5, 3.5) if network_type == 'Completa' else (2.5, 2.5); plt.figure(figsize=fig_size); n = red.number_of_nodes()
    color_map = {'C': '#DEDEEE', 'D': '#9B9ACA', 'H': '#7976B8', 'K': '#201F3D', 'Unknown': '#808080'}
    node_colors = [color_map.get(estrategias_finales.get(nodo, 'Unknown'), '#808080') for nodo in red.nodes()]
    print(f"  Calculando layout para red {network_type} ({n} nodos)..."); start_time = time.time()
    try: pos = nx.spring_layout(red, seed=42, k=0.6/np.sqrt(n) if n>0 else 0.1, iterations=50 if network_type=='Barabási-Albert' else 30)
    except Exception: pos = nx.circular_layout(red)
    print(f"  Layout calculado en {time.time() - start_time:.2f} seg.")
    print(f"  Dibujando red final {network_type}..."); node_size = 10 if network_type == 'Completa' else 10; nx.draw_networkx_nodes(red, pos, node_color=node_colors, node_size=node_size, linewidths=0.1, edgecolors='face')
    edge_alpha = 0.5 if network_type == 'Completa' else 0.5; edge_width = 0.5 if network_type == 'Completa' else 0.3; nx.draw_networkx_edges(red, pos, edge_color="#A7A7A7", alpha=edge_alpha, width=edge_width)
    title_net_part = f"{network_type}{network_params_str}" if network_type=='Barabási-Albert' else network_type; plt.title(f"Red de {titulo}\n{title_net_part} - Homofilia: {int(float(homophily_str) * 100)}\\% \n{cm_title_str}"); plt.axis('off')
    os.makedirs(os.path.dirname(filename), exist_ok=True); save_format = "png" if network_type == 'Completa' else "svg"; save_dpi = 150 if network_type == 'Completa' else 300; plt.savefig(filename, format=save_format, dpi=save_dpi, bbox_inches='tight'); print(f"  Gráfica de red final guardada como '{filename}' (Formato: {save_format})"); plt.close()
    return pos

def graficar_subgrafo_k(red_original, estrategias_finales, pos_original, titulo, network_type, network_params_str, homophily_str, cm_title_str, filename):
    nodos_k = [nodo for nodo, strat in estrategias_finales.items() if strat == 'K']
    if not nodos_k: print(f"  No nodos 'K' p/ subgrafo en {titulo}."); return None
    print(f"  Creando subgrafo 'K' ({len(nodos_k)} nodos)..."); subgrafo = red_original.subgraph(nodos_k)
    pos_k = {nodo: pos_original[nodo] for nodo in nodos_k if nodo in pos_original}
    if len(pos_k) != len(nodos_k): print(" Adv: No pos para todos K."); nodos_a_dibujar = list(pos_k.keys()); subgrafo = red_original.subgraph(nodos_a_dibujar);
    else: nodos_a_dibujar = nodos_k
    if not nodos_a_dibujar: return None
    plt.style.use(['science', 'ieee']); plt.figure(figsize=(2.5, 2.5))
    nx.draw_networkx_nodes(subgrafo, pos_k, nodelist=nodos_a_dibujar, node_color='#201F3D', node_size=10, linewidths=0.1, edgecolors='face')
    nx.draw_networkx_edges(subgrafo, pos_k, edge_color="#201F3D", alpha=0.6, width=0.4)
    title_net_part = f"{network_type}{network_params_str}" if network_type=='Barabási-Albert' else network_type; plt.title(f"Red de nodos corruptos (K) de {titulo}\n{title_net_part} - Homofilia: {int(float(homophily_str) * 100)}\\% \n{cm_title_str}"); plt.axis('off')
    os.makedirs(os.path.dirname(filename), exist_ok=True); plt.savefig(filename, format="svg", dpi=300, bbox_inches='tight'); print(f"  Gráfica de subgrafo K guardada como '{filename}'"); plt.close()
    return filename

# --- Función Principal ---
def simular_y_analizar(
    nombre_pais, params_juego, num_nodos,
    network_type, network_params,
    cm_percentage, cm_placement_type,
    cm_strategies, non_cm_strategies,
    K, homophily_strength, rondas, seed,
    output_dir_base,
    flags
    ):
    # 1. Crear nombre de directorio y archivo descriptivo y sanitizado
    net_label = f"BA_m{network_params['m']}" if network_type == 'Barabási-Albert' else 'Comp'
    cm_place_label = cm_placement_type[:3].upper()
    cm_strat_label = "".join(sorted(cm_strategies))
    sanitized_pais = re.sub(r'[^\w-]', '', nombre_pais.lower())
    run_name = f"{sanitized_pais}_{net_label}_CM{cm_percentage}p{cm_place_label}{cm_strat_label}_H{int(homophily_strength*100)}p"
    output_dir_run = os.path.join(output_dir_base, run_name)
    os.makedirs(output_dir_run, exist_ok=True)
    output_path_base = os.path.join(output_dir_run, run_name) # Base para todos los archivos de esta corrida

    # 2. Ejecutar Simulación
    print(f"\n--- Iniciando Simulación: {run_name} ---")
    cm_size_actual = int(round((cm_percentage / 100) * num_nodos))
    historia, red_final, estrategias_finales = ejecutar_simulacion_corrupcion(
        num_nodos=num_nodos, network_type=network_type, network_params=network_params, cm_size_actual=cm_size_actual, cm_placement_type=cm_placement_type,
        cm_strategies=cm_strategies, non_cm_strategies=non_cm_strategies, params_juego=params_juego, K=K, homophily_strength=homophily_strength,
        rondas=rondas, seed=seed
    )

    # 3. Graficar Evolución
    titulo_evol = f"Evolución de {nombre_pais} \n {network_type} (m={network_params}) - Homofilia: {int(float(homophily_strength) * 100)}\\% \n Masa crítica: {cm_placement_type} ({cm_strat_label})"
    graficar_evolucion_corrupcion(
        historia_fracciones=historia,
        titulo=nombre_pais,
        network_type_str=network_type,
        homophily_str=str(homophily_strength),
        cm_title_str=f"{cm_placement_type} ({cm_strat_label})",
        filename=f"{output_path_base}_evolucion.svg"
    )
    print(f"  Gráfica de evolución guardada en: {output_dir_run}")

    # 4. Procesar Red Final y Subred K
    pos = None
    if flags['graficar_redes']:
        print("  Calculando layout para red final...")
        try: pos = nx.spring_layout(red_final, seed=seed, k=0.6/np.sqrt(num_nodos) if num_nodos>0 else 0.1, iterations=30)
        except: pos = nx.circular_layout(red_final)
        titulo_red = f"Red de {nombre_pais} / {net_label} / {cm_percentage}% CM {cm_place_label} ({cm_strat_label}) / H={homophily_strength*100}%"
        graficar_red_completa(red_final, estrategias_finales, pos, titulo_red, output_path_base)
        print(f"  Gráfica de red final guardada en: {output_dir_run}")

    nodos_k = [n for n, s in estrategias_finales.items() if s == 'K']
    if not nodos_k:
        print("  No se encontraron nodos 'K' para analizar.")
        return

    subgrafo_k = red_final.subgraph(nodos_k).copy()
    
    # 5. Guardar y Analizar Subred K
    if flags['guardar_subgrafo_k_datos']:
        path_datos_graphml = f"{output_path_base}_subgrafoK.graphml"
        nx.write_graphml(subgrafo_k, path_datos_graphml)
        print(f"  Datos de subgrafo K guardados en: {path_datos_graphml}")

    if flags['analizar_subgrafo_k_metricas']:
        print("  Iniciando análisis de métricas de subred K...")
        output_dir_analisis_k = os.path.join(output_dir_run, "analisis_subred_K")
        nombre_base_analisis_k = f"analisis_{run_name}"
        output_path_base_analisis_k = os.path.join(output_dir_analisis_k, nombre_base_analisis_k)
        analizar_subred_k(subgrafo_k, output_path_base_analisis_k)

# --- Ejemplo de Uso ---
if __name__ == "__main__":
    # --- Parámetros de Simulación y Control ---
    NODOS = 100
    RONDAS = 10
    SEED_GLOBAL = 42
    RUIDO_K = 0.0
    OUTPUT_DIR_GLOBAL = f'Resultados_Simulacion'
    
    # --- Flags para controlar qué se ejecuta ---
    flags_ejecucion = {
        "graficar_redes": True,
        "guardar_subgrafo_k_datos": True,
        "analizar_subgrafo_k_metricas": True,
    }

    # --- Selección de Escenarios a Ejecutar ---
    PAISES_A_SIMULAR = ['México']
    config_paises = { 'Dinamarca': PARAMS_DINAMARCA, 'Singapur': PARAMS_SINGAPUR, 'México': PARAMS_MEXICO }
    config_cm_percentages = [50] # [10, 50, 90]
    niveles_homofilia = [0.0] # [0.0, 0.8]
    config_redes = [ {'network_type': 'Barabási-Albert', 'network_params': {'m': 1}} ]
    config_cm_placement = ['Nodos centrales'] # ['Nodos centrales', 'Nodos perifericos', 'Nodos aleatorios']
    config_cm_strategies = [ {'label': 'SoloK', 'cm_strats': ['K'], 'non_cm_strats': ['C', 'D', 'H']} ]

    # --- Lógica de Ejecución ---
    paises_filtrados = {p: config_paises[p] for p in PAISES_A_SIMULAR if p in config_paises}

    for pais, params_juego_pais in paises_filtrados.items():
        for cm_percentage in config_cm_percentages:
            for cm_strat_config in config_cm_strategies:
                for homophily_level in niveles_homofilia:
                    for sim_config_red in config_redes:
                        for cm_place_type in config_cm_placement:
                            simular_y_analizar(
                                 nombre_pais=pais, params_juego=params_juego_pais, num_nodos=NODOS,
                                 network_type=sim_config_red['network_type'], network_params=sim_config_red['network_params'],
                                 cm_percentage=cm_percentage, cm_placement_type=cm_place_type,
                                 cm_strategies=cm_strat_config['cm_strats'], non_cm_strategies=cm_strat_config['non_cm_strats'],
                                 K=RUIDO_K, homophily_strength=homophily_level, rondas=RONDAS, seed=SEED_GLOBAL,
                                 output_dir_base=OUTPUT_DIR_GLOBAL,
                                 flags=flags_ejecucion
                             )
    print(f"\nTodas las simulaciones y análisis completados. Resultados en '{OUTPUT_DIR_GLOBAL}'.")

: 

### Integración de las funciones de analisis de las subredes corruptas con el código del modelo refinado

(Este código es el funcional del modelo refinado de Scatà, se necesita modificar para integrar las funciones de analisis de redes corruptas)

In [2]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
import scienceplots
import os
from collections import Counter
import time
# Opcional para mostrar SVG en notebooks
try:
    from IPython.display import SVG, display, Image
except ImportError:
    SVG, display, Image = None, None, None

# --- Parámetros Juego y Funciones Auxiliares (sin cambios) ---
PARAMS_DINAMARCA = {'r': 2.5, 's': 1, 't': 3, 'p': 8, 'c': 0.2, 'q': 11, 'd': 0.5}
PARAMS_SINGAPUR =  {'r': 2, 's': 1, 't': 4, 'p': 10, 'c': 0.2, 'q': 15, 'd': 0.2}
PARAMS_MEXICO =    {'r': 2, 's': 1, 't': 4, 'p': 10, 'c': 2, 'q': 5, 'd': 4}

def calcular_pago_individual_corrupcion(strat_i, strat_j, params):
    r, s, t, p, c, q, d = params['r'], params['s'], params['t'], params['p'], params['c'], params['q'], params['d']
    payoffs = { ('C', 'C'): r, ('C', 'D'): -s, ('C', 'H'): r, ('C', 'K'): -s, ('D', 'C'): t, ('D', 'D'): 0, ('D', 'H'): t-p, ('D', 'K'): -p, ('H', 'C'): r, ('H', 'D'): -s-c, ('H', 'H'): r, ('H', 'K'): -s-d, ('K', 'C'): t, ('K', 'D'): -c, ('K', 'H'): t-q, ('K', 'K'): -q-d }
    return payoffs.get((strat_i, strat_j), 0)

def actualizar_estrategia_corrupcion_red_homofilia(red, nodo, estrategias, recompensas_totales, K, homophily_strength):
    estrategia_actual = estrategias[nodo]; recompensa_total_actual = recompensas_totales[nodo]
    vecinos = list(red.neighbors(nodo))
    if not vecinos: return estrategia_actual
    vecino_elegido = random.choice(vecinos); estrategia_vecino = estrategias[vecino_elegido]; recompensa_total_vecino = recompensas_totales[vecino_elegido]
    probabilidad_adoptar_base = 0.0
    try:
        if K <= 1e-9: probabilidad_adoptar_base = 1.0 if recompensa_total_vecino > recompensa_total_actual else 0.0
        else: delta_recompensa_scaled = np.clip((recompensa_total_actual - recompensa_total_vecino) / K, -700, 700); probabilidad_adoptar_base = 1 / (1 + np.exp(delta_recompensa_scaled))
    except (OverflowError, FloatingPointError): probabilidad_adoptar_base = 1.0 if recompensa_total_vecino > recompensa_total_actual else 0.0
    if estrategia_actual == estrategia_vecino: probabilidad_adoptar_final = probabilidad_adoptar_base * (1 + homophily_strength)
    else: probabilidad_adoptar_final = probabilidad_adoptar_base * (1 - homophily_strength)
    probabilidad_adoptar_final = np.clip(probabilidad_adoptar_final, 0.0, 1.0)
    return estrategia_vecino if random.random() < probabilidad_adoptar_final else estrategia_actual

def calcular_centralidad(red):
    try: centrality = nx.eigenvector_centrality_numpy(red); print("  Usando centralidad de Eigenvector.")
    except Exception as e: print(f"  Advertencia: Eigenvector falló ({e}). Usando Grado."); centrality = nx.degree_centrality(red)
    return centrality

def seleccionar_nodos_cm(centrality_dict, cm_size, placement_type):
    if cm_size > len(centrality_dict): raise ValueError(f"CM size ({cm_size}) > num nodos.")
    if cm_size == 0: return []
    reverse_sort = (placement_type == 'Nodos centrales')
    nodos_ordenados = sorted(centrality_dict, key=centrality_dict.get, reverse=reverse_sort)
    if placement_type in ['Nodos centrales', 'Nodos perifericos']: cm_nodes = nodos_ordenados[:cm_size]
    elif placement_type == 'Nodos aleatorios': cm_nodes = random.sample(list(centrality_dict.keys()), cm_size)
    else: raise ValueError(f"Colocación CM desconocida: {placement_type}")
    return cm_nodes

def inicializar_estrategias_cm_flexible(n, cm_nodes, cm_strategies, non_cm_strategies, seed=None):
    if seed is not None: random.seed(seed)
    estrategias = {}; non_cm_nodes = list(set(range(n)) - set(cm_nodes))
    if not cm_strategies: raise ValueError("cm_strategies vacía.")
    if not non_cm_strategies: raise ValueError("non_cm_strategies vacía.")
    random.shuffle(cm_nodes); random.shuffle(non_cm_nodes)
    for i, nodo in enumerate(cm_nodes): estrategias[nodo] = cm_strategies[i % len(cm_strategies)]
    for i, nodo in enumerate(non_cm_nodes): estrategias[nodo] = non_cm_strategies[i % len(non_cm_strategies)]
    return dict(sorted(estrategias.items()))

# --- Ejecución Simulación Modificada ---
def ejecutar_simulacion_corrupcion(
    num_nodos=100,
    network_type='Completa', network_params=None,
    cm_size=50, cm_percentage=50, # <--- Nuevo parámetro para el título
    cm_placement_type='Nodos aleatorios',
    cm_strategies = ['C', 'H'], non_cm_strategies = ['D', 'K'],
    params_juego=PARAMS_MEXICO, K=0.1, homophily_strength=0.0,
    rondas=15, seed=None
    ):
    if seed is not None: random.seed(seed); np.random.seed(seed)
    print(f"Creando red tipo '{network_type}'...")
    network_params_str_out = ""
    if network_type == 'Completa': red = nx.complete_graph(num_nodos)
    elif network_type == 'Barabási-Albert':
        if network_params is None: network_params = {}
        m = network_params.get('m', 2)
        if m >= num_nodos: m = max(1, num_nodos // 2)
        red = nx.barabasi_albert_graph(num_nodos, m, seed=seed); network_params_str_out = f"_(m={m})"
    else: raise ValueError(f"Tipo de red desconocido: {network_type}")
    print(f"Red creada: {red.number_of_nodes()} nodos, {red.number_of_edges()} enlaces.")
    print(f"Calculando centralidad...")
    centrality = calcular_centralidad(red)
    print(f"Seleccionando {cm_size} nodos CM ({cm_placement_type})...")
    cm_nodes = seleccionar_nodos_cm(centrality, cm_size, cm_placement_type)
    print(f"Inicializando estrategias: CM={cm_strategies}, No-CM={non_cm_strategies} con seed={seed}...")
    estrategias = inicializar_estrategias_cm_flexible(num_nodos, cm_nodes, cm_strategies, non_cm_strategies, seed=seed)
    counts_init = Counter(estrategias.values()); print(f"  Estrategias iniciales: {counts_init}")
    historia_fracciones = {'C': [], 'D': [], 'H': [], 'K': []}; nodos_lista = list(red.nodes())
    print(f"Iniciando {rondas} rondas...")
    for r in range(rondas):
        recompensas_totales = {nodo: 0 for nodo in nodos_lista}
        for nodo_i in nodos_lista:
            strat_i = estrategias[nodo_i]; vecinos_i = list(red.neighbors(nodo_i))
            if not vecinos_i: continue
            for nodo_j in vecinos_i: strat_j = estrategias[nodo_j]; recompensas_totales[nodo_i] += calcular_pago_individual_corrupcion(strat_i, strat_j, params_juego)
        siguientes_estrategias = {}; nodos_actualizar = nodos_lista[:]; random.shuffle(nodos_actualizar)
        for nodo in nodos_actualizar: siguientes_estrategias[nodo] = actualizar_estrategia_corrupcion_red_homofilia(red, nodo, estrategias, recompensas_totales, K, homophily_strength)
        estrategias = siguientes_estrategias
        conteo_actual = Counter(estrategias.values())
        for strat_key in ['C', 'D', 'H', 'K']: fraccion = conteo_actual.get(strat_key, 0) / num_nodos if num_nodos > 0 else 0; historia_fracciones[strat_key].append(fraccion)
    print("Simulación completada.")
    cm_strat_str_file = "".join(sorted(cm_strategies))
    if cm_placement_type == 'Nodos centrales': cm_placement_label_file = "Cen"
    elif cm_placement_type == 'Nodos perifericos': cm_placement_label_file = "Per"
    elif cm_placement_type == 'Nodos aleatorios': cm_placement_label_file = "Ale"
    else: cm_placement_label_file = cm_placement_type[:3].lower()
    # Usar cm_percentage en el nombre del archivo
    cm_info_str = f"_CM{cm_percentage}"
    cm_title_str = f"Masa crítica: {cm_percentage}\\% - {cm_placement_type} ({','.join(cm_strategies)})"
    # --- FIN CORRECCIÓN ---
    return historia_fracciones, red, estrategias, network_type, network_params_str_out, cm_info_str, cm_title_str

# --- Funciones de Graficado (sin cambios) ---
def graficar_evolucion_corrupcion(historia_fracciones, titulo, network_type_str, homophily_str, cm_title_str, filename):
    plt.style.use(['science', 'ieee']); plt.figure(figsize=(2.5, 2.5))
    colores = {'C': '#DEDEEE', 'D': '#9B9ACA', 'H': '#7976B8', 'K': '#201F3D'}
    estilos = {'C': '-', 'D': '--', 'H': ':', 'K': '-.'}; nombres_legenda = {'C': '(C) C. Coop', 'D': '(D) C. no Coop', 'H': '(H) P. Honrado', 'K': '(K) P. Corrupto'}
    rondas_totales = len(historia_fracciones['C']); eje_x = range(rondas_totales)
    for strat in ['C', 'D', 'H', 'K']: plt.plot(eje_x, historia_fracciones[strat], label=nombres_legenda[strat], color=colores[strat], linestyle=estilos[strat], linewidth=1)
    plt.title(f"Evolución de {titulo}\n{network_type_str} - Homofilia: {int(float(homophily_str) * 100)}\\% \n{cm_title_str}")
    plt.ylim(0, 1.0); plt.xlim(0, rondas_totales -1 if rondas_totales > 1 else 1); plt.legend(fontsize=7, loc='best'); plt.grid(True, linestyle='--', alpha=0.6)
    os.makedirs(os.path.dirname(filename), exist_ok=True); plt.savefig(filename, format="svg", dpi=300, bbox_inches='tight'); print(f"Gráfica de evolución guardada como '{filename}'"); plt.close()

def graficar_red_final(red, estrategias_finales, titulo, network_type, network_params_str, homophily_str, cm_title_str, filename):
    if not isinstance(red, nx.Graph) or not red.nodes(): return None
    plt.style.use(['science', 'ieee']); fig_size = (3.5, 3.5) if network_type == 'Completa' else (2.5, 2.5); plt.figure(figsize=fig_size); n = red.number_of_nodes()
    color_map = {'C': '#DEDEEE', 'D': '#9B9ACA', 'H': '#7976B8', 'K': '#201F3D', 'Unknown': '#808080'}
    node_colors = [color_map.get(estrategias_finales.get(nodo, 'Unknown'), '#808080') for nodo in red.nodes()]
    print(f"  Calculando layout para red {network_type} ({n} nodos)..."); start_time = time.time()
    try: pos = nx.spring_layout(red, seed=42, k=0.6/np.sqrt(n) if n>0 else 0.1, iterations=50 if network_type=='Barabási-Albert' else 30)
    except Exception: pos = nx.circular_layout(red)
    print(f"  Layout calculado en {time.time() - start_time:.2f} seg.")
    print(f"  Dibujando red final {network_type}..."); node_size = 10 if network_type == 'Completa' else 10; nx.draw_networkx_nodes(red, pos, node_color=node_colors, node_size=node_size, linewidths=0.1, edgecolors='face')
    edge_alpha = 0.5 if network_type == 'Completa' else 0.5; edge_width = 0.5 if network_type == 'Completa' else 0.3; nx.draw_networkx_edges(red, pos, edge_color="#A7A7A7", alpha=edge_alpha, width=edge_width)
    title_net_part = f"{network_type}{network_params_str}" if network_type=='Barabási-Albert' else network_type; plt.title(f"Red de {titulo}\n{title_net_part} - Homofilia: {int(float(homophily_str) * 100)}\\% \n{cm_title_str}"); plt.axis('off')
    os.makedirs(os.path.dirname(filename), exist_ok=True); save_format = "png" if network_type == 'Completa' else "svg"; save_dpi = 150 if network_type == 'Completa' else 300; plt.savefig(filename, format=save_format, dpi=save_dpi, bbox_inches='tight'); print(f"  Gráfica de red final guardada como '{filename}' (Formato: {save_format})"); plt.close()
    return pos

def graficar_subgrafo_k(red_original, estrategias_finales, pos_original, titulo, network_type, network_params_str, homophily_str, cm_title_str, filename):
    nodos_k = [nodo for nodo, strat in estrategias_finales.items() if strat == 'K']
    if not nodos_k: print(f"  No nodos 'K' p/ subgrafo en {titulo}."); return None
    print(f"  Creando subgrafo 'K' ({len(nodos_k)} nodos)..."); subgrafo = red_original.subgraph(nodos_k)
    pos_k = {nodo: pos_original[nodo] for nodo in nodos_k if nodo in pos_original}
    if len(pos_k) != len(nodos_k): print(" Adv: No pos para todos K."); nodos_a_dibujar = list(pos_k.keys()); subgrafo = red_original.subgraph(nodos_a_dibujar);
    else: nodos_a_dibujar = nodos_k
    if not nodos_a_dibujar: return None
    plt.style.use(['science', 'ieee']); plt.figure(figsize=(2.5, 2.5))
    nx.draw_networkx_nodes(subgrafo, pos_k, nodelist=nodos_a_dibujar, node_color='#201F3D', node_size=10, linewidths=0.1, edgecolors='face')
    nx.draw_networkx_edges(subgrafo, pos_k, edge_color="#201F3D", alpha=0.6, width=0.4)
    title_net_part = f"{network_type}{network_params_str}" if network_type=='Barabási-Albert' else network_type; plt.title(f"Red de nodos corruptos (K) de {titulo}\n{title_net_part} - Homofilia: {int(float(homophily_str) * 100)}\\% \n{cm_title_str}"); plt.axis('off')
    os.makedirs(os.path.dirname(filename), exist_ok=True); plt.savefig(filename, format="svg", dpi=300, bbox_inches='tight'); print(f"  Gráfica de subgrafo K guardada como '{filename}'"); plt.close()
    return filename

# --- Función Principal Modificada ---
def simular_y_graficar_pais(
    nombre_pais, params_juego, num_nodos,
    network_type, network_params,
    cm_size, cm_percentage, # <--- Nuevo parámetro de porcentaje
    cm_placement_type, cm_strategies, non_cm_strategies,
    K, homophily_strength, rondas, seed,
    output_dir="Resultados_Corrupcion",
    graficar_redes=True, graficar_subgrafo_k_flag=True
    ):
    cm_strats_str_label = "".join(sorted(cm_strategies))
    if cm_placement_type == 'Nodos centrales': cm_placement_label = "Cen"
    elif cm_placement_type == 'Nodos perifericos': cm_placement_label = "Per"
    elif cm_placement_type == 'Nodos aleatorios': cm_placement_label = "Ale"
    else: cm_placement_label = cm_placement_type[:3].lower()
    print(f"\n--- Sim: {nombre_pais} / {network_type}{network_params} / CM:{cm_placement_label}({cm_percentage}% {cm_strats_str_label}) / H={homophily_strength} ---")
    historia, red_final, estrategias_finales, net_type_used, net_params_str_out, cm_info_str, cm_title_str = ejecutar_simulacion_corrupcion(
        num_nodos=num_nodos, network_type=network_type, network_params=network_params,
        cm_size=cm_size, cm_percentage=cm_percentage, 
        cm_placement_type=cm_placement_type,
        cm_strategies=cm_strategies, non_cm_strategies=non_cm_strategies,
        params_juego=params_juego, K=K, homophily_strength=homophily_strength,
        rondas=rondas, seed=seed
    )
    homophily_file_str = f"_h{homophily_strength:.2f}".replace('.', '')
    homophily_title_str = f"{homophily_strength:.2f}"
    nombre_base = f"{nombre_pais.lower().replace(' ', '_')}_{net_type_used}{net_params_str_out}{cm_info_str}{homophily_file_str}"
    network_title_str = f"{net_type_used}{net_params_str_out.replace('_', ' ')}"
    path_grafica_evolucion = os.path.join(output_dir, f"evolucion_{nombre_base}.svg")
    graficar_evolucion_corrupcion(historia, nombre_pais, network_title_str, homophily_title_str, cm_title_str, path_grafica_evolucion)
    pos_calculado = None; path_grafica_red_final = None
    if graficar_redes:
        format_red = 'png' if net_type_used == 'Completa' else 'svg'
        path_grafica_red_final = os.path.join(output_dir, f"red_final_{nombre_base}.{format_red}")
        pos_calculado = graficar_red_final(red_final, estrategias_finales, nombre_pais, net_type_used, net_params_str_out, homophily_title_str, cm_title_str, path_grafica_red_final)
    path_grafica_subgrafo_k = None
    if graficar_subgrafo_k_flag:
        if pos_calculado is None and graficar_redes:
             print("  Calculando pos solo para subgrafo K...")
             try: pos_calculado = nx.spring_layout(red_final, seed=seed, k=0.6/np.sqrt(num_nodos) if num_nodos>0 else 0.1, iterations=50 if net_type_used=='Barabási-Albert' else 30)
             except Exception: pos_calculado = None
        if pos_calculado:
            path_grafica_subgrafo_k = os.path.join(output_dir, f"subgrafo_k_{nombre_base}.svg")
            graficar_subgrafo_k(red_final, estrategias_finales, pos_calculado, nombre_pais, net_type_used, net_params_str_out, homophily_title_str, cm_title_str, path_grafica_subgrafo_k)
        else: print("  No pos, omitiendo subgrafo K.")
    print(f"--- Simulación completada ---")
    return path_grafica_evolucion, path_grafica_red_final, path_grafica_subgrafo_k

# --- Ejemplo de Uso (Modificado) ---
if __name__ == "__main__":
    # --- Parámetros de Simulación y Control ---
    NODOS = 1000
    RUIDO_K = 0.0
    RONDAS = 100
    SEED_GLOBAL = 42
    OUTPUT_DIR_GLOBAL = f'N{NODOS}_I{RONDAS}_CM5a95'
    GRAFICAR_REDES_FINALES = True
    GRAFICAR_SUBGRAFO_K = True

    # --- Selección de Escenarios a Ejecutar ---
    PAISES_A_SIMULAR = ['México', 'Dinamarca', 'Singapur']  # Puedes ajustar esta lista para simular solo algunos países ['México', 'Dinamarca', 'Singapur']
    config_paises = {
        'Dinamarca': PARAMS_DINAMARCA,
        'Singapur': PARAMS_SINGAPUR,
        'México': PARAMS_MEXICO
    }
    
    config_cm_strategies = [
        #{'label': 'Coop', 'cm_strats': ['C', 'H'], 'non_cm_strats': ['D', 'K']},
        #{'label': 'NoCoop', 'cm_strats': ['D', 'K'], 'non_cm_strats': ['C', 'H']},
        {'label': 'SoloK', 'cm_strats': ['K'], 'non_cm_strats': ['C', 'D', 'H']}
        #{'label': 'KvH', 'cm_strats': ['H'], 'non_cm_strats': [ 'K']},
        #{'label': 'KvCH', 'cm_strats': ['H', 'C'], 'non_cm_strats': [ 'K']}
    ]
    
    config_cm_placement = ['Nodos centrales'] #['Nodos centrales', 'Nodos perifericos', 'Nodos aleatorios'] # Puedes ajustar esto según tus necesidades
    
    # **** NUEVO: Lista de porcentajes de CM a probar ****
    CM_PORCENTAJES_A_PROBAR = [
        #5, 10, 
        #15,
        #20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 
        95
        ] # En porcentajes (e.g., 15 para 15%)

    niveles_homofilia = [
        0.0,
        #0.5, 0.8, 0.9, 1
        ] # Niveles de homofilia a probar [0.0, 0.1, 0.2, 0.3, 0.4, 0.5] etc.
    config_redes = [
        #{'network_type': 'Completa', 'network_params': {}},
        {'network_type': 'Barabási-Albert', 'network_params': {'m': 1}}
    ]

    # --- Lógica de Ejecución ---
    paths_graficas = {}
    paises_filtrados = {pais: config_paises[pais] for pais in PAISES_A_SIMULAR if pais in config_paises}

    # Añadir nuevo bucle para porcentajes de CM
    for cm_percentage in CM_PORCENTAJES_A_PROBAR:
        cm_size = round((cm_percentage / 100) * NODOS)
        if cm_size == 0 and NODOS > 0:
            print(f"Omitiendo CM %={cm_percentage} porque resulta en 0 nodos de masa crítica.")
            continue

        print(f"\n==================== CM PORCENTAJE = {cm_percentage}% (Tamaño={cm_size}) ====================")
        for pais, params_juego_pais in paises_filtrados.items():
            print(f"\n==================== PAÍS: {pais.upper()} ====================")
            for cm_strat_config in config_cm_strategies:
                cm_label = cm_strat_config['label']
                cm_strats_list = cm_strat_config['cm_strats']
                non_cm_strats_list = cm_strat_config['non_cm_strats']
                print(f"\n========== MASA CRÍTICA = {cm_label} ({','.join(cm_strats_list)}) ==========")
                for homophily_level in niveles_homofilia:
                    print(f"\n===== HOMOFILIA = {homophily_level:.2f} =====")
                    for sim_config_red in config_redes:
                        net_type = sim_config_red['network_type']
                        net_params = sim_config_red['network_params']
                        print(f"\n===== RED: {net_type} {net_params} =====")
                        for cm_place_type in config_cm_placement:
                            print(f"\n===== CM Placement: {cm_place_type} =====")

                            if cm_place_type == 'Nodos centrales': cm_place_label_dir = "Cen"
                            elif cm_place_type == 'Nodos perifericos': cm_place_label_dir = "Per"
                            elif cm_place_type == 'Nodos aleatorios': cm_place_label_dir = "Ale"
                            else: cm_place_label_dir = cm_place_type[:3].lower()

                            net_params_str_dir = "".join([f'_{k}{v}' for k,v in net_params.items()]) if net_params else ""
                            homophily_str_dir = f"_h{homophily_level:.2f}".replace('.', '')
                            cm_strat_str_dir_label = f"_CM{cm_label}"
                            cm_percent_str_dir = f"_P{cm_percentage}" # Para el nombre del directorio

                            # Estructura de directorios: Pais / CM_Tipo / CM_Porcentaje / Red_Tipo+Params / Placement+Homofilia
                            output_subdir_base = os.path.join(OUTPUT_DIR_GLOBAL, pais, f"CM_{cm_label}", f"P{cm_percentage}")
                            output_subdir = os.path.join(output_subdir_base, f"Red_{net_type}{net_params_str_dir}", f"{cm_place_label_dir}{homophily_str_dir}")

                            clave_base = f"{pais}_{net_type}{net_params_str_dir}_{cm_place_label_dir}_{cm_strat_str_dir_label}{cm_percent_str_dir}{homophily_str_dir}"

                            path_evol, path_red, path_sub_k = simular_y_graficar_pais(
                                 pais, params_juego_pais, NODOS, net_type, net_params,
                                 cm_size, cm_percentage, # <-- Pasar ambos, tamaño y %
                                 cm_place_type,
                                 cm_strats_list, non_cm_strats_list,
                                 RUIDO_K, homophily_level, RONDAS, SEED_GLOBAL, output_subdir,
                                 graficar_redes=GRAFICAR_REDES_FINALES,
                                 graficar_subgrafo_k_flag=GRAFICAR_SUBGRAFO_K
                             )
                            paths_graficas[f"{clave_base}_evol"] = path_evol
                            if path_red: paths_graficas[f"{clave_base}_red"] = path_red
                            if path_sub_k: paths_graficas[f"{clave_base}_subgrafo_k"] = path_sub_k

    print(f"\nTodas las simulaciones completadas. Resultados en '{OUTPUT_DIR_GLOBAL}'.")


==================== CM PORCENTAJE = 95% (Tamaño=950) ====================

==================== PAÍS: MÉXICO ====================

========== MASA CRÍTICA = SoloK (K) ==========

===== HOMOFILIA = 0.00 =====

===== RED: Barabási-Albert {'m': 1} =====

===== CM Placement: Nodos centrales =====

--- Sim: México / Barabási-Albert{'m': 1} / CM:Cen(95% K) / H=0.0 ---
Creando red tipo 'Barabási-Albert'...
Red creada: 1000 nodos, 999 enlaces.
Calculando centralidad...
  Usando centralidad de Eigenvector.
Seleccionando 950 nodos CM (Nodos centrales)...
Inicializando estrategias: CM=['K'], No-CM=['C', 'D', 'H'] con seed=42...
  Estrategias iniciales: Counter({'K': 950, 'D': 17, 'C': 17, 'H': 16})
Iniciando 100 rondas...
Simulación completada.
Gráfica de evolución guardada como 'N1000_I100_CM5a95\México\CM_SoloK\P95\Red_Barabási-Albert_m1\Cen_h000\evolucion_méxico_Barabási-Albert_(m=1)_CM95_h000.svg'
  Calculando layout para red Barabási-Albert (1000 nodos)...
  Layout calculado en 17.45 seg.
